Data explore: 


Feature selection:

first, simply try step-wise feature selection for the basic model.
For the selected features, try the pairs of combination.

then, use tree-based algorithm rank important variables

combine tree-based algorithm and step-wise solutions

do the same steps with Customers, since it is not in testing set.

At last, apply linear regression to test the result.



In [ ]:
train = read.csv(file = '../input/457-engineered/train_engineered (2).csv', header = TRUE)
test = read.csv(file = '../input/457-engineered/test_engineered (2).csv', header = TRUE)

In [ ]:
train_sub = train[train$Sales > 0, ]#only use positive Sales in training data
testOpen = (test$Open == 1)
test_sub = test[testOpen, ] #only needs to do prediction for open days

In [ ]:
colnames(train_sub)

In [ ]:
library(e1071)    
skewness(train_sub$Sales)

In [ ]:
hist(train_sub$Sales, breaks= 100) 

In [ ]:
 skewness(log(train_sub$Sales))

In [ ]:
hist(log(train_sub$Sales), breaks= 100) 

# Feature selection for Sales

# Variable importance with tree-based algorithm

In [ ]:
library(caret)
set.seed(100)
#rPartMod <- train(Sales ~(factor(Store) + factor(DayOfWeek) + factor(year) + factor(month) + day + StoreType + CompetitionDistance )^2, data=train_sub, method="rpart")
# for less computational cost
rPartMod <- train(Sales ~factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(year) + factor(month) + day + StoreType + CompetitionDistance, data=train_sub, method="rpart")
rpartImp <- varImp(rPartMod)
print(rpartImp)

It shows that Customers, CompetitionDistance, Promo, factor(Store), factor(DayOfWeek) should be selected

# Stepwise Feature Selection

In [ ]:
model_try = lm(Sales ~(factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(year) + factor(month) + day + StoreType + CompetitionDistance), data = train_sub)
step(model_try, direction="both", k = 2)

factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(year) + factor(month) + day are selected. Try to look at their combinations

In [ ]:
# ignore customers first
# factor on categorical variables with more than 2 attributes, according to histogram with each vairables
model_try = lm(Sales ~(factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(year) + factor(month) + day)^2, data = train_sub)

In [ ]:
# Step-wise feature selection
step(model_try, direction="both", k = 2)

Result model from step-wise is: 

lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + 
    Promo + SchoolHoliday + factor(year) + factor(month) + day + 
    factor(Store):factor(DayOfWeek) + factor(Store):Customers + 
    factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(year) + 
    factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + 
    factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + 
    factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
    Customers:Promo + Customers:SchoolHoliday + Customers:factor(year) + 
    Customers:factor(month) + Customers:day + Promo:SchoolHoliday + 
    Promo:factor(year) + Promo:factor(month) + Promo:day + SchoolHoliday:factor(year) + 
    SchoolHoliday:factor(month) + factor(year):factor(month) + 
    factor(year):day + factor(month):day, data = train_sub)

lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + factor(year) + 
    factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):factor(year) + 
    factor(Store):day + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
    factor(DayOfWeek):day + factor(year):day + 
    factor(month):day, data = train_sub)


In [ ]:
library(MLmetrics)
# self test RMSPE
smp_size <- floor(0.75 * nrow(train_sub))

## set the seed to make your partition reproducible
set.seed(123)
train_ind <- sample(seq_len(nrow(train_sub)), size = smp_size)

train_1 <- train_sub[train_ind, ]
test_1 <- train_sub[-train_ind, ]

library(MLmetrics)
#lm.mod = lm(Sales ~ factor(Store) * factor(DayOfWeek), data = train_sub)
#lm.mod = lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):factor(year) + factor(Store):day + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + factor(DayOfWeek):day + factor(year):day + factor(month):day, data = train_sub)
# made imporvement of 0.06

#Customers, CompetitionDistance, Promo


#lm.mod = lm(Sales ~ factor(Store) * factor(DayOfWeek) + Customers + CompetitionDistance + Promo, data = train_sub)
#0.096

#lm.mod = lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):factor(year) + factor(Store):day + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + factor(DayOfWeek):day + factor(year):day + factor(month):day + Customers + CompetitionDistance + Promo, data = train_sub)
#0.0920480092545085, current best model

lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + 
    Promo + SchoolHoliday + factor(year) + factor(month) + day + 
    factor(Store):factor(DayOfWeek) + factor(Store):Customers + 
    factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(year) + 
    factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + 
    factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + 
    factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
    Customers:Promo + Customers:SchoolHoliday + Customers:factor(year) + 
    Customers:factor(month) + Customers:day + Promo:SchoolHoliday + 
    Promo:factor(year) + Promo:factor(month) + Promo:day + SchoolHoliday:factor(year) + 
    SchoolHoliday:factor(month) + factor(year):factor(month) + 
    factor(year):day + factor(month):day + CompetitionDistance, data = train_sub)
#0.0920480092545085, current best model

y_pred = predict(lm.mod, test_1[ , !(names(test_1) %in% c('Sales'))])
y_true = as.numeric(as.vector(unlist(test_1['Sales'])))

RMSPE(y_pred = y_pred, y_true = y_true)

Final feature selections: lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):factor(year) + factor(Store):day + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + factor(DayOfWeek):day + factor(year):day + factor(month):day + Customers + CompetitionDistance + Promo, data = train_sub)

But we do not have data for customers in test set, need to predict it.

# Feature selection for Customers

In [ ]:
# by similar steps
library(caret)
set.seed(100)
#rPartMod <- train(Sales ~(factor(Store) + factor(DayOfWeek) + factor(year) + factor(month) + day + StoreType + CompetitionDistance )^2, data=train_sub, method="rpart")
# for less computational cost
rPartMod <- train(Customers ~factor(Store) + factor(DayOfWeek) + Promo + SchoolHoliday + factor(year) + factor(month) + day + StoreType + CompetitionDistance, data=train_sub, method="rpart")
rpartImp <- varImp(rPartMod)
print(rpartImp)

CompetitionDistance, factor(Store), Promo, StoreType, factor(DayOfWeek) should be selected

# Stepwise Feature Selection

In [ ]:
# ignore customers first
# factor on categorical variables with more than 2 attributes, according to histogram with each vairables
model_try = lm(Customers ~(factor(Store) + factor(DayOfWeek) + Promo + SchoolHoliday + factor(year) + factor(month) + day + StoreType + CompetitionDistance)^2, data = train_sub)

In [ ]:
# Step-wise feature selection
step(model_try, direction="both", k = 2)

There is NA with factor(DayOfWeek):Promo, factor(year) : factor(month)

Including previous selections, The final model is: 
lm(formula = Customers ~ factor(Store) + factor(DayOfWeek) + 
    Promo + SchoolHoliday + factor(year) + factor(month) + day + 
    factor(Store):factor(DayOfWeek) + factor(Store):Promo + factor(Store):SchoolHoliday + 
    factor(Store):factor(year) + factor(Store):factor(month) + 
    factor(Store):day + factor(DayOfWeek):SchoolHoliday + 
    factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
    factor(DayOfWeek):day + Promo:factor(year) + Promo:factor(month) + 
    Promo:day + SchoolHoliday:factor(month) + SchoolHoliday:day + factor(month):day + CompetitionDistance + StoreType, data = train_sub)

In [ ]:
library(MLmetrics)
# self test RMSPE
smp_size <- floor(0.75 * nrow(train_sub))

## set the seed to make your partition reproducible
set.seed(123)
train_ind <- sample(seq_len(nrow(train_sub)), size = smp_size)

train_1 <- train_sub[train_ind, ]
test_1 <- train_sub[-train_ind, ]

library(MLmetrics)
'''
lm.mod = lm(formula = Customers ~ factor(Store) + factor(DayOfWeek) + 
    Promo + SchoolHoliday + factor(year) + factor(month) + day + 
    factor(Store):factor(DayOfWeek) + factor(Store):Promo + factor(Store):SchoolHoliday + 
    factor(Store):factor(year) + factor(Store):factor(month) + 
    factor(Store):day + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + 
    factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
    factor(DayOfWeek):day + Promo:factor(year) + Promo:factor(month) + 
    Promo:day + SchoolHoliday:factor(month) + SchoolHoliday:day + 
    factor(year):factor(month) + factor(month):day + CompetitionDistance + StoreType, data = train_sub)
# 0.139843998880705
'''
lm(formula = Customers ~ factor(Store) + factor(DayOfWeek) + 
    Promo + SchoolHoliday + factor(year) + factor(month) + day + 
    factor(Store):factor(DayOfWeek) + factor(Store):Promo + factor(Store):SchoolHoliday + 
    factor(Store):factor(year) + factor(Store):factor(month) + 
    factor(Store):day + factor(DayOfWeek):SchoolHoliday + 
    factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
    factor(DayOfWeek):day + Promo:factor(year) + Promo:factor(month) + 
    Promo:day + SchoolHoliday:factor(month) + SchoolHoliday:day + factor(month):day + CompetitionDistance + StoreType, data = train_sub)
# 0.139843998880705

y_pred = predict(lm.mod, test_1[ , !(names(test_1) %in% c('Sales, Customers'))])
y_true = as.numeric(as.vector(unlist(test_1['Customers'])))

RMSPE(y_pred = y_pred, y_true = y_true)

In [ ]:
lm.mod_customer = lm(formula = Customers ~ factor(Store) + factor(DayOfWeek) + 
    Promo + SchoolHoliday + factor(year) + factor(month) + day + 
    factor(Store):factor(DayOfWeek) + factor(Store):Promo + factor(Store):SchoolHoliday + 
    factor(Store):factor(year) + factor(Store):factor(month) + 
    factor(Store):day + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + 
    factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
    factor(DayOfWeek):day + Promo:factor(year) + Promo:factor(month) + 
    Promo:day + SchoolHoliday:factor(month) + SchoolHoliday:day + 
    factor(year):factor(month) + factor(month):day + CompetitionDistance + StoreType, data = train_sub)
test_sub$Customers = predict(lm.mod_customer, test_sub)

lm.mod = lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):factor(year) + factor(Store):day + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + factor(DayOfWeek):day + factor(year):day + factor(month):day + Customers + CompetitionDistance + Promo, data = train_sub)
lm.pred = predict(lm.mod, test_sub)
outDat = data.frame(ID = 1:nrow(test), Sales = rep(0,nrow(test))) 
outDat$Sales[testOpen] = lm.pred

#Write the prediction to an output file
write.csv(outDat, file = 'linear_regression_0316.csv', row.names = FALSE)

In [ ]:
test_sub$Customers

In [ ]:
outDat

0.0920480092545085, 0.139843998880705

Variable importance with tree-based algorithm

In [ ]:
library(caret)
set.seed(100)
#rPartMod <- train(Sales ~(factor(Store) + factor(DayOfWeek) + factor(year) + factor(month) + day + StoreType + CompetitionDistance )^2, data=train_sub, method="rpart")
# for less computational cost
rPartMod <- train(Sales ~factor(Store) + factor(DayOfWeek) + factor(year) + factor(month) + day + StoreType + CompetitionDistance, data=train_sub, method="rpart")
rpartImp <- varImp(rPartMod)
print(rpartImp)

In [ ]:
Lasso regression feature selection

In [ ]:
library(glmnet)
lasso.cv.out = cv.glmnet(as.matrix(trainDat[,-9]), trainDat$lpsa, alpha = 1) 

Apply Relative Importance from Linear Regression

In [ ]:
library(relaimpo)

# Build linear regression model

lmMod <- lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):factor(year) + factor(Store):day + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + factor(DayOfWeek):day + factor(year):day + factor(month):day, data = train_sub)

# calculate relative importance
relImportance <- calc.relimp(lmMod, type = "lmg", rela = F)  

# Sort
cat('Relative Importances: \n')
sort(round(relImportance$lmg, 3), decreasing=TRUE)

In [ ]:
library(caret)
set.seed(100)
rPartMod <- train(Class ~ ., data=trainData, method="rpart")
rpartImp <- varImp(rPartMod)
print(rpartImp)

library(MASS)
library(caret)
# library(randomForest)

set.seed(123)
# Set up repeated k-fold cross-validation
train.control <- trainControl(method = "cv", number = 10)

# Train the model
step.model <- train(formula = Sales ~ factor(Store) + factor(DayOfWeek) + factor(year) + 
                    factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):factor(year) + 
                    factor(Store):day + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
                    factor(DayOfWeek):day + factor(year):factor(month) + factor(year):day + 
                    factor(month):day, data = train_sub,
                    method = "rvmLinear", 
                    # tuneGrid = data.frame(nvmax = 1:55),
                    trControl = train.control
                    )
step.model$results

step.model$bestTune